In [1]:
import pandas as pd
import numpy as np

In [2]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [3]:
transactions_train=pd.read_csv("drive/My Drive/Colab Notebooks/дз1/transactions_train.csv")

In [4]:
train_target=pd.read_csv("drive/My Drive/Colab Notebooks/дз1/train_target.csv")

In [5]:
transactions_train.head()

,client_id,trans_date,small_group,amount_rur
0,33172,6,4,71.463
1,33172,6,35,45.017
2,33172,8,11,13.887
3,33172,9,11,15.983
4,33172,10,11,21.341


* client_id - уникальный идентификатор клиента
* trans_date - дата совершения транзакции
* small_group - категория покупки
* amount_rur - сумма транзакции

In [6]:
train_target.head(5)

,client_id,bins
0,24662,2
1,1046,0
2,34089,2
3,34848,1
4,47076,3


* client_id - уникальный идентификатор клиента, соответствует полю client_id из транзакций
* bins - целевая переменная, которую нужно предсказать, это категория возраста клиента

Посчитаем по каждому клиенту самые простые аггрегационные признаки.

In [7]:
agg_features=transactions_train.groupby('client_id')['amount_rur'].agg(['sum','mean','std','min','max']).reset_index()

In [8]:
agg_features.head()

,client_id,sum,mean,std,min,max
0,4,28404.121,39.450168,73.511624,0.043,1341.802
1,6,15720.739,21.535259,26.200397,0.045,315.781
2,7,53630.036,69.379089,253.261383,0.043,4505.971
3,10,34419.365,48.752642,63.191701,0.045,654.893
4,11,26789.404,32.991877,107.395139,0.388,2105.058


Посчитаем для каждого клиента количество транзакций по каждой категории.

In [9]:
counter_df_train=transactions_train.groupby(['client_id','small_group'])['amount_rur'].count()

In [10]:
cat_counts_train=counter_df_train.reset_index().pivot(index='client_id', \
                                                      columns='small_group',values='amount_rur')

In [11]:
cat_counts_train=cat_counts_train.fillna(0)

In [12]:
cat_counts_train.columns=['small_group_'+str(i) for i in cat_counts_train.columns]

In [13]:
cat_counts_train.head()

,small_group_0,small_group_1,small_group_2,small_group_3,small_group_4,small_group_5,small_group_6,small_group_7,small_group_8,small_group_9,small_group_10,small_group_11,small_group_12,small_group_13,small_group_14,small_group_15,small_group_16,small_group_17,small_group_18,small_group_19,small_group_20,small_group_21,small_group_22,small_group_23,small_group_24,small_group_25,small_group_26,small_group_27,small_group_28,small_group_29,small_group_30,small_group_31,small_group_32,small_group_33,small_group_34,small_group_35,small_group_36,small_group_37,small_group_38,small_group_39,...,small_group_162,small_group_163,small_group_164,small_group_165,small_group_166,small_group_167,small_group_168,small_group_169,small_group_170,small_group_171,small_group_172,small_group_173,small_group_174,small_group_175,small_group_176,small_group_177,small_group_178,small_group_179,small_group_180,small_group_181,small_group_182,small_group_183,small_group_184,small_group_185,small_group_186,small_group_187,small_group_188,small_group_189,small_group_190,small_group_191,small_group_192,small_group_193,small_group_195,small_group_196,small_group_197,small_group_198,small_group_199,small_group_200,small_group_202,small_group_203
client_id,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
4,0.0,447.0,1.0,44.0,93.0,0.0,0.0,0.0,1.0,13.0,1.0,19.0,0.0,0.0,0.0,37.0,0.0,0.0,5.0,0.0,2.0,0.0,0.0,3.0,6.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,2.0,0.0,9.0,0.0,12.0,0.0,0.0,3.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
6,2.0,397.0,0.0,172.0,10.0,0.0,0.0,0.0,0.0,6.0,3.0,9.0,0.0,0.0,0.0,103.0,0.0,0.0,1.0,0.0,2.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,3.0,0.0,1.0,0.0,0.0,0.0,2.0,0.0,0.0,5.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
7,2.0,79.0,5.0,27.0,19.0,1.0,0.0,2.0,1.0,39.0,0.0,187.0,136.0,5.0,0.0,15.0,3.0,0.0,64.0,3.0,0.0,3.0,2.0,7.0,15.0,55.0,0.0,0.0,6.0,0.0,0.0,1.0,1.0,2.0,8.0,5.0,16.0,5.0,0.0,3.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
10,12.0,309.0,1.0,71.0,65.0,0.0,0.0,0.0,3.0,19.0,0.0,58.0,0.0,0.0,0.0,40.0,0.0,0.0,6.0,2.0,0.0,0.0,0.0,3.0,26.0,11.0,0.0,0.0,6.0,0.0,1.0,2.0,0.0,0.0,8.0,0.0,27.0,0.0,0.0,12.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
11,2.0,423.0,0.0,59.0,23.0,3.0,0.0,0.0,0.0,10.0,4.0,107.0,0.0,1.0,0.0,7.0,1.0,0.0,22.0,0.0,1.0,1.0,18.0,3.0,14.0,17.0,3.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,2.0,10.0,14.0,0.0,0.0,1.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


Далее соединим все файлы в один датафрейм с таргетом.

In [14]:
train=pd.merge(train_target,agg_features,on='client_id')

In [15]:
train=pd.merge(train,cat_counts_train.reset_index(),on='client_id')

In [16]:
train.head()

,client_id,bins,sum,mean,std,min,max,small_group_0,small_group_1,small_group_2,small_group_3,small_group_4,small_group_5,small_group_6,small_group_7,small_group_8,small_group_9,small_group_10,small_group_11,small_group_12,small_group_13,small_group_14,small_group_15,small_group_16,small_group_17,small_group_18,small_group_19,small_group_20,small_group_21,small_group_22,small_group_23,small_group_24,small_group_25,small_group_26,small_group_27,small_group_28,small_group_29,small_group_30,small_group_31,small_group_32,...,small_group_162,small_group_163,small_group_164,small_group_165,small_group_166,small_group_167,small_group_168,small_group_169,small_group_170,small_group_171,small_group_172,small_group_173,small_group_174,small_group_175,small_group_176,small_group_177,small_group_178,small_group_179,small_group_180,small_group_181,small_group_182,small_group_183,small_group_184,small_group_185,small_group_186,small_group_187,small_group_188,small_group_189,small_group_190,small_group_191,small_group_192,small_group_193,small_group_195,small_group_196,small_group_197,small_group_198,small_group_199,small_group_200,small_group_202,small_group_203
0,24662,2,30254.011,34.774725,72.037354,0.074,1227.314,0.0,174.0,2.0,64.0,33.0,0.0,0.0,0.0,1.0,3.0,0.0,92.0,365.0,0.0,0.0,11.0,0.0,0.0,20.0,0.0,0.0,4.0,3.0,3.0,9.0,16.0,4.0,0.0,4.0,0.0,0.0,4.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,1046,0,42548.570,52.015367,106.540962,0.550,1210.506,1.0,187.0,61.0,47.0,13.0,1.0,0.0,0.0,2.0,8.0,1.0,27.0,3.0,0.0,1.0,79.0,0.0,0.0,142.0,0.0,2.0,0.0,4.0,2.0,5.0,4.0,3.0,0.0,6.0,1.0,0.0,5.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,34089,2,26842.816,34.325852,59.927450,0.043,782.641,0.0,372.0,0.0,72.0,37.0,10.0,0.0,0.0,0.0,17.0,0.0,47.0,9.0,0.0,0.0,49.0,15.0,1.0,6.0,0.0,2.0,2.0,1.0,5.0,26.0,21.0,0.0,0.0,3.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,34848,1,15773.126,16.160990,14.224936,0.043,109.590,0.0,359.0,1.0,0.0,41.0,0.0,0.0,0.0,0.0,38.0,0.0,116.0,0.0,0.0,0.0,306.0,0.0,0.0,45.0,0.0,1.0,0.0,0.0,0.0,0.0,4.0,0.0,0.0,3.0,0.0,0.0,1.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,47076,3,12488.375,15.929050,35.473591,0.432,541.165,0.0,378.0,0.0,150.0,44.0,0.0,0.0,0.0,0.0,122.0,0.0,33.0,0.0,1.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0,0.0,2.0,2.0,8.0,31.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


Теперь подгрузим тестовые данные для того, чтобы сделать предсказание. Проделаем с ними те же самые манипуляции, как и с обучающими данными.

In [17]:
transactions_test=pd.read_csv("drive/My Drive/Colab Notebooks/дз1/transactions_test.csv")

test_id=pd.read_csv("drive/My Drive/Colab Notebooks/дз1/transactions_test.csv")

In [18]:
agg_features_test=transactions_test.groupby('client_id')['amount_rur'].agg(['sum','mean','std','min','max']).reset_index()

In [19]:
agg_features_test.head()

,client_id,sum,mean,std,min,max
0,0,17036.127,19.163247,40.561700,0.065,595.339
1,1,34748.964,47.666617,89.489278,0.298,1181.221
2,2,51188.069,68.433247,152.093601,0.043,2837.682
3,3,47975.203,67.857430,318.651653,0.043,6135.652
4,5,20059.100,21.803370,34.258433,0.043,439.902


In [20]:
counter_df_test=transactions_test.groupby(['client_id','small_group'])['amount_rur'].count()

In [21]:
cat_counts_test=counter_df_test.reset_index().pivot(index='client_id', columns='small_group',values='amount_rur')

In [22]:
cat_counts_test=cat_counts_test.fillna(0)

In [23]:
cat_counts_test.columns=['small_group_'+str(i) for i in cat_counts_test.columns]

In [24]:
cat_counts_test.head()

,small_group_0,small_group_1,small_group_2,small_group_3,small_group_4,small_group_5,small_group_6,small_group_7,small_group_8,small_group_9,small_group_10,small_group_11,small_group_12,small_group_13,small_group_14,small_group_15,small_group_16,small_group_17,small_group_18,small_group_19,small_group_20,small_group_21,small_group_22,small_group_23,small_group_24,small_group_25,small_group_26,small_group_27,small_group_28,small_group_29,small_group_30,small_group_31,small_group_32,small_group_33,small_group_34,small_group_35,small_group_36,small_group_37,small_group_38,small_group_39,...,small_group_162,small_group_163,small_group_164,small_group_165,small_group_166,small_group_167,small_group_168,small_group_169,small_group_170,small_group_171,small_group_172,small_group_173,small_group_174,small_group_175,small_group_176,small_group_177,small_group_178,small_group_179,small_group_180,small_group_181,small_group_182,small_group_183,small_group_184,small_group_185,small_group_186,small_group_187,small_group_188,small_group_189,small_group_190,small_group_191,small_group_192,small_group_193,small_group_194,small_group_195,small_group_196,small_group_197,small_group_198,small_group_200,small_group_201,small_group_202
client_id,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
0,0.0,226.0,1.0,36.0,9.0,0.0,0.0,0.0,2.0,20.0,0.0,238.0,81.0,0.0,0.0,21.0,9.0,2.0,82.0,0.0,0.0,0.0,0.0,2.0,6.0,48.0,1.0,0.0,2.0,1.0,1.0,3.0,1.0,6.0,0.0,4.0,16.0,0.0,0.0,1.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,30.0,326.0,0.0,40.0,56.0,0.0,0.0,0.0,0.0,60.0,0.0,2.0,0.0,0.0,0.0,137.0,3.0,0.0,0.0,0.0,2.0,1.0,1.0,1.0,22.0,10.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,13.0,19.0,0.0,1.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,21.0,242.0,1.0,50.0,48.0,4.0,0.0,6.0,1.0,21.0,7.0,58.0,28.0,6.0,0.0,20.0,9.0,2.0,31.0,0.0,3.0,2.0,3.0,7.0,21.0,13.0,2.0,0.0,12.0,1.0,1.0,9.0,8.0,1.0,20.0,7.0,30.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,0.0,156.0,83.0,48.0,31.0,2.0,0.0,1.0,2.0,27.0,0.0,148.0,0.0,0.0,6.0,35.0,20.0,3.0,8.0,0.0,1.0,9.0,2.0,4.0,13.0,10.0,1.0,0.0,4.0,0.0,0.0,6.0,0.0,1.0,26.0,0.0,0.0,5.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
5,16.0,398.0,1.0,23.0,25.0,0.0,0.0,0.0,5.0,29.0,1.0,62.0,1.0,0.0,0.0,94.0,20.0,0.0,4.0,2.0,0.0,1.0,5.0,3.0,25.0,53.0,0.0,0.0,0.0,0.0,111.0,0.0,0.0,0.0,1.0,4.0,9.0,0.0,0.0,1.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [25]:
test=pd.merge(test_id,agg_features_test,on='client_id')

В тесте не было некоторых категорий трат, поэтому для того, чтобы обучить модель, нам нужно объединить пространство признаков и train и test.

In [26]:
common_features=list(set(train.columns).intersection(set(test.columns)))

In [27]:
y_train=train['bins']
X_train=train[common_features]
X_test=test[common_features]

In [28]:
X = X_train
y = y_train

In [33]:
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split


In [34]:
X_train, X_test, y_train, y_test = train_test_split(X ,y , test_size=1/3)

model = LogisticRegression()
model.fit(X_train, y_train)
y_pred = model.predict(X_test)
print(model.score(X_test, y_test))


0.3298


/usr/local/lib/python3.7/dist-packages/sklearn/linear_model/_logistic.py:940: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  extra_warning_msg=_LOGISTIC_SOLVER_CONVERGENCE_MSG)
